# **Multiblock I/O 예시**



```sql
=================================================================
call      count    cpu   elapsed   disk   query   current    rows
-------- ------ ------ --------- ------ ------- --------- -------     
Parse         1   0.00      0.00      0       0         0       0
Execute       1   0.00      0.00      0       0         0       0
Fetch         2   0.26      0.26     64      69         0       1
-------- ------ ------ --------- ------ ------- --------- -------
total         4   0.26      0.26     64      69         0       1
=================================================================
```


- 위 Call 통계 결과
- 버퍼 캐시에서 69개 블록을 읽으면서 64개는 디스크에서 읽음.
- 버퍼 캐시 히트율은 7.24%.
- 디스크에서 읽은 블록 수가 64개라고 I/O Call도 동일하지 않음.
- 동일하거나 그보다 작을 수도 있음.

# **I/O Call을 통해 데이터파일에서 버퍼 캐시에 적재하는 방식**

**Single Block I/O**

- 한번의 I/O Call에 하나의 데이터 블록만 읽어 메모리에 적재하는 것.
- 인덱스를 통해 테이블을 액세스할 때, 기본적으로 인덱스와 테이블 블록 모두 이 방식을 사용.
- 인덱스 블록간 논리적 순서
  - 인덱스 리프 블록끼리 이중 연결 리스트(Double Linked List) 구조로 연결된 순서.
  - 물리적으로 데이터파일에 저장된 순서와 다르다.
  - 물리적으로 한 Extent에 속한 블록들을 I/O Call 발생 시점에 같이 적재하여 올렸을 때, 그 블록들이 논리적 순서로는 한참 뒤쪽에 위치할 수 있으므로, 실제 사용되지 못한 채 버퍼상에서 밀려 날 수도 있으므로 Singl Block I/O방식이 효율적.
- Index Range Scan 뿐 아니라 Index Full Scan시에도 논리적인 순서에 따라 Single Block I/O방식으로 읽음.
- Index Fast Full Scan은 Multiblock I/O 방식을 사용.

**Multiblock I/O**

- I/O Call이 필요한 시점에 인접한 블록들을 같이 읽어 메모리에 적재하는 것.
- Full Scan과 같이 저장된 순서에 따라 읽을 때는 그릇(한번 I/O에 1MB의 '그릇'을 사용)이 허용하는 범위 내에서 인접한 블락을 읽는 것이 유리.
  - 인접한 블락은 한 익스텐트 범위 내의 블락을 말함.
  - 즉, Multiblock I/O을 하더라도 한 익스텐트 범위를 넘지 못함.
- Multiblock I/O 단위는 db_file_muliblock_read_count 파라미터에 의해 결정.
- Index Fast Full Scan은 Multiblock I/O 방식을 사용.

# **I/O 요청시 대기 이벤트 종류**

- 서버 프로세스는 Disk에서 블록을 읽어야 하는 시점마다 I/O 서브시스템에 I/O 요청을 하고 대기 상태에 빠짐.

**db file sequential read**

- Single Block I/O방식으로 I/O를 요청할 때 발생.
- Singl Block I/O방식으로 읽은 블록들은 LRU 리스트 상 MRU쪽 end로 연결되므로 한번 적재되면 버퍼 캐시에 비교적 오래 머뭄.

**db file scattered read**
- Multiblock I/O방식으로 I/O를 요청할 때 발생.
- Multiblock I/O방식으로 읽은 블록들은 LRU 리스트에서 LRU쪽 end로 연결되므로 적재되고 얼마 지나지 않아 버퍼캐시에서 밀려남.
